## Загрузка библиотек

In [ ]:
!pip install -qq transformers
!pip install watermark

In [ ]:
import watermark
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers,sklearn.linear_model, plotly

In [ ]:
import transformers
import torch

import io
import time
import pickle
import csv
import os

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from matplotlib import rc
from pylab import rcParams

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pd.set_option('display.max_colwidth', 200)


Подключаем Google Drive для локального хранения данных

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Настраиваем отображение графиков (для работы в Google Colab)

In [ ]:
# !pip install cufflinks --upgrade 

import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js?x49997"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## Подготовка обучающей выборки

Загружаем размеченную выборку

In [ ]:
!gdown --id 1MSfU7VGOlXSGdObF5hSE7XDnZ5frz0zc

Изучаем данные:

In [ ]:
df = pd.read_csv('Training_data.csv', 
                  header=0)

In [ ]:
df = df.drop_duplicates(subset=['Sentence'])

In [ ]:
df = df[:1700]

In [ ]:
df['word_count'] = df.Sentence.apply(lambda x: len(x.split()))

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.Score.unique()

In [ ]:
max(df.word_count)

## EDA

Визуализируем данные:

In [ ]:
class_names = ['Нейтральная', 'Положительная', 'Отрицательная']

In [ ]:
ax = sns.countplot(df.Score, order =df.Score.value_counts().index)
colors = ["#87CEFA", "#90EE90", "#FA8072"]
sns.set_palette(sns.color_palette(colors))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)
plt.xlabel('Тональность отчетов');
plt.ylabel('Количество');
ax.set_xticklabels(class_names);

Обучающая выборка получилась несбалансированной – доля предложений с нейтральной тональностью значительно превышает остальные. Однако это отражает специфику нефинансовой отчетности, в которой также преобладает нейтральная тональность, поэтому наполнение выборки было решено оставить без изменений.  

Распределение длины предложений по категориям тональности

In [ ]:
configure_plotly_browser_state()

df[df['Score']==1]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='red',
    yTitle='count',
    title='Positive Text Length Distribution')

df[df['Score']==-1]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    color='green',
    yTitle='count',
    title='Negative Text Length Distribution')

df[df['Score']==0]['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='text length',
    linecolor='black',
    yTitle='count',
    title='Neutral Text Length Distribution')

## Обучение BERT

### Токенизация предложений

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tkz = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
sample_txt = 'Развитие компании проходит в условиях повышенной конкуренции'

In [ ]:
tokens_tkz = tkz.tokenize(sample_txt)
tokens_tkz

In [ ]:
text_encoded = tkz.encode(tokens_tkz)
text_encoded

In [ ]:
text_decode = tkz.decode(text_encoded)
text_decode 

### Загрузка предобученной модели

In [ ]:
from transformers import AutoModelWithLMHead

In [ ]:
model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

### Обучение модели на размеченной выборке

In [ ]:
df.Sentence = df.Sentence.map(lambda x: str(x).lower()) # переводим символы в нижний регистр

In [ ]:
df['tokenized'] = df.Sentence.apply((lambda x: tkz.encode(tkz.tokenize(x))))

In [ ]:
df.head()

Так как обучение BERT требует много памяти, разобьем обучающую выборку на несколько частей и будем последовательно обучать на них модель

In [ ]:
df_dict = dict()
for i in range(0,len(df),200):
  df_dict['{}'.format(i)] = df[i:i+200]

df_dict.keys()

In [ ]:
padded = np.array([i + [0]*(max_len-len(i)) for i in df_1.tokenized.values])

In [ ]:
np.array(padded).shape

In [ ]:
np.array(padded)[2]

Собираем обработанные данные

In [ ]:
def train_model(df, features_list):

  # Токенизируем предложения
  df['tokenized'] = df.Sentence.apply((lambda x: tkz.encode(tkz.tokenize(x))))

  # Вычисляем длину наибольшего токена
  max_len = 0
  for index, i in df.tokenized.iteritems():
      if len(i) > max_len:
          max_len = len(i)

  # Создаем массив и заполняем его токенами
  padded = np.array([i + [0]*(max_len-len(i)) for i in df.tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  # Обучаем модель

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)  

  # Сохраняем признаки
  features = last_hidden_states[0][:,0,:].numpy()
  features_list.append(features)

  return None

In [ ]:
features_list_1 = []

for i in range(0,800,200):
  prepare_df(df_dict["{}".format(i)], features_list=features_list_1)
  time.sleep(5)

In [ ]:
features_list_2 = []

for i in range(800,1200,200):
  prepare_df(df_dict["{}".format(i)], features_list=features_list_2)
  time.sleep(5)

In [ ]:
features_list_3 = []

for i in range(1200,1700,200):
  prepare_df(df_dict["{}".format(i)], features_list=features_list_3)
  time.sleep(5)

In [ ]:
features_1 = np.vstack(features_list_1)
features_2 = np.vstack(features_list_2)
features_3 = np.vstack(features_list_3)

Локально сохраняем сгенерированные признаки:

In [ ]:
filename_1 = '/content/gdrive/My Drive/bert_features_1'
filename_2 = '/content/gdrive/My Drive/bert_features_2'
filename_3 = '/content/gdrive/My Drive/bert_features_3'

np.save(file=filename,arr=features_1)
np.save(file=filename,arr=features_2)
np.save(file=filename,arr=features_3)

Загружаем признаки и объединяем в один список

In [ ]:
!cp "/content/gdrive/My Drive/bert_features_1.npy" "bert_features_1.npy"
!cp "/content/gdrive/My Drive/bert_features_2.npy" "bert_features_2.npy"
!cp "/content/gdrive/My Drive/bert_features_3.npy" "bert_features_3.npy"

In [ ]:
feat_1 = np.load('bert_features_1.npy')
feat_2 = np.load('bert_features_2.npy')
feat_3 = np.load('bert_features_3.npy')

In [ ]:
features = np.vstack([feat_1,feat_2,feat_3])
print('Количество признаков в обучающей выборке:', features.shape[1])

### Train-test split

Разбиваем обучающую выборку на тренировочную и тестовую

In [ ]:
labels = df.Score

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25)

## Выбор классификатора

Обучим классификаторы, которые будут предсказывать тональность предложений по сгенерированным моделью BERT признакам. Опробуем несколько классификаторов и выберем тот, который покажет наибольшую точность на контрольной выборке

#### Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent')

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#### Логистическая регрессия

##### Обучаем классификатор

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_clf = LogisticRegression(max_iter=1000, C=4.0)
lr_clf.fit(train_features, train_labels)

In [ ]:
accuracy = lr_clf.score(test_features, test_labels)
print('LR Accuracy:', accuracy)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
labels_test = test_labels.to_list()

In [ ]:
labels_pred = lr_clf.predict(test_features).tolist()

In [ ]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

##### Сохраняем результаты

In [ ]:
filename = 'lr_model.sav'
pickle.dump(lr_clf, open(filename, 'wb'))

#### SVM

##### Обучаем классификатор

In [ ]:
from sklearn import svm

In [ ]:
svm_clf = svm.SVC(kernel='linear', C=4.0) 

In [ ]:
svm_clf.fit(train_features, train_labels)

In [ ]:
svm_clf.fit(train_features, train_labels)

In [ ]:
labels_test = test_labels.to_list()

In [ ]:
accuracy = svm_clf.score(test_features, test_labels)
print('SVM Accuracy:', accuracy)

In [ ]:
labels_pred = svm_clf.predict(test_features).tolist()

In [ ]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Naive Bayes

##### Обучаем классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb_clf = GaussianNB()

In [ ]:
gnb_clf.fit(train_features, train_labels)

In [ ]:
accuracy = gnb_clf.score(test_features, test_labels)
print('GNB Accuracy:', accuracy)

In [ ]:
labels_pred = gnb_clf.predict(test_features).tolist()

In [ ]:
labels_test = test_labels.to_list()

In [ ]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Decision Trees

##### Обучаем классификатор

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

In [ ]:
dtc_clf = DecisionTreeClassifier()

In [ ]:
dtc_clf.fit(train_features, train_labels)

In [ ]:
accuracy = dtc_clf.score(test_features, test_labels)
print('Decision Tree Accuracy:', accuracy)

In [ ]:
labels_pred = dtc_clf.predict(test_features).tolist()

In [ ]:
labels_test = test_labels.to_list()

In [ ]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

#### Random Forest

##### Обучаем классификатор 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf=RandomForestClassifier(n_estimators=100)

In [ ]:
rf_clf.fit(train_features, train_labels)

In [ ]:
accuracy = rf_clf.score(test_features, test_labels)
print('Decision Tree Accuracy:', accuracy)

Random Forest оказался наилучшим классификатором, показав точность 0.83 на контрольной выборке.

In [ ]:
labels_pred = rf_clf.predict(test_features).tolist()

In [ ]:
labels_test = test_labels.to_list()

In [ ]:
class_names = ['Отрицательная', 'Нейтральная', 'Положительная']
print(classification_report(y_true=labels_test, 
                            y_pred=labels_pred,
                            target_names=class_names))

In [ ]:
df.shape

In [ ]:
rf_clf.fit(train_features, train_labels)

##### Сохраняем результаты

In [ ]:
filename = 'rf_model.sav'
pickle.dump(rf_clf, open(filename, 'wb'))

##### Загружаем модель

In [ ]:
!cp "/content/gdrive/My Drive/rf_model.sav" 'rf_model.sav'

In [ ]:
filename = 'rf_model.sav'
rf_clf = pickle.load(open(filename, 'rb'))
rf_clf

### Матрица ошибок

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('Правильная тональность')
  plt.xlabel('Предсказанная тональность');

In [ ]:
cm = confusion_matrix(labels_test, labels_pred)
cm

In [ ]:
df_pred = pd.DataFrame(data=list(zip(labels_test,labels_pred)), columns=['True','Predicted'])
df_pred

In [ ]:
class_names = ['Негативная', 'Нейтральная', 'Положительная']

In [ ]:
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

## Предсказываем тональность

#### По тексту

In [ ]:
txt = [
       'Компания нацелена на предупреждение, сокращение и минимизацию последствий разливов нефти и нефтепродуктов',
       'В Компании сформирована и развивается система оперативного реагирования на разливы нефти и нефтепродуктов, их локализации и ликвидации с целью минимизации экологических последствий, в том числе влияния на водные ресурсы',
       ' В части «Раскрытия информации» место Компании в рейтинге экологической ответственности поднялось на восемь пунктов, с десятого на второе место по сравнению с предыдущим периодом',
       'Главный актив Компании – это высокопрофессиональный персонал, мотивированный на эффективную работу',
       'Повышение эффективности труда остается одним из ключевых приоритетов Компании. В рамках реализации этой задачи в 2018 году актуализированы внутрикорпоративные методики расчета показателей производительности труда по Компании в целом, по основным бизнес-блокам и Обществам Группы основных бизнес-блоков',
       'В Компании разработан перечень мероприятий по росту производительности труда в Компании. Мероприятия включены в Долгосрочную программу развития Компании, отчет по исполнению которой происходит на ежегодной основе',
       'Компания подвержена влиянию множества присущих нефтегазовой отрасли рисков, основными из которых являются снижение цен на нефть и нефтепродукты и рост цен на приобретаемое сырье и услуги',
       'Компания ведет мониторинг законопроектов, что позволяет заблаговременно оценить последствия предлагаемых изменений и учесть их в своих планах. Компания имеет обширный опыт реализации проектов в области добычи и переработки углеводородного сырья, а также обладает финансовыми, материально-техническими и кадровыми ресурсами, необходимыми для выполнения обязательств по лицензионным соглашениям',
       'Характерными для Компании финансовыми рисками являются валютный, процентный, инфляционный, кредитный риски и риск ликвидности. Данные риски могут негативно повлиять на финансовые результаты деятельности Компании вследствие роста расходов, обесценения активов, снижения рентабельности и денежного потока Компании',
       'В региональном конкурсе реализованных проектов в области энергосбережения и повышения энергоэффективности ENES–2018 Компания признана победителем в двух номинациях: «Лучшая реализованная комплексная программа в ТЭК по популяризации энергосбережения и повышения энергоэффективности» и «Эффективная система управления в области энергосбережения и повышения энергоэффективности на предприятиях ТЭК»',
       'Важное значение Компания придает программам, направленным на развитие и поддержку массового спорта, физического развития сотрудников и их детей',
       'В целях повышения качества комплектования квалифицированными кадрами требуемых профессий и нужной квалификации в Компании проведено 7539 тестирований кандидатов на трудоустройство по 100 рабочим профессиям на базе компьютерных классов',
       'Компания нацелена на предупреждение, сокращение и минимизацию последствий разливов нефти',
       'В блоке «Нефтепереработка и нефтегазохимия» реализуется комплекс программ по обеспечению целостности оборудования и исключению аварийных ситуаций с неблагоприятными экологическими последствиями',
       'Компания является одним из крупнейших работодателей в России'
]

In [ ]:
txt_tk = [tkz.encode(tkz.tokenize(x)) for x in txt]

In [ ]:
max_len = 0
for i in txt_tk:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

In [ ]:
padded = np.array([i + [0]*(max_len-len(i)) for i in txt_tk])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features

In [ ]:
preds = list(rf_clf.predict(features))
preds

In [ ]:
classes_dict = {-1:'negative', 0:'neutral', 1:'positive'}
classes_dict

In [ ]:
preds_ls = list(zip(txt, preds))

In [ ]:
for k,v in preds_ls:
  print(classes_dict.get(v), k, sep=' : ')

#### По нефинансовым отчетам

##### Предварительная обработка текста


In [ ]:
INPUT_DIR ='/content/gdrive/My Drive/'
REPORTS_DIR = '/content/reports/'

try:
  os.mkdir(REPORTS_DIR)
except FileExistsError:
      print(REPORTS_DIR+' already exists')

In [ ]:
def clean_df(file_r, df):

  # Считываем текстовый файл с отчетом в таблицу, одна строка таблицы - один абзац текста
  df = pd.read_csv(file_r, sep='\n',header=None, names=['Sentence'],quoting=csv.QUOTE_NONE) 

  # Вычисляем длину строк, оставляем те, длина которых превышает 3 слова
  df['len'] = df.Sentence.apply(lambda x: len(x.split()))
  df = df[df.len >=3]

  # Разбиваем абзацы на отдельные предложения
  new_df = pd.DataFrame(df.Sentence.str.split('.').tolist(), index=df.index).stack()
  new_df = new_df.reset_index()
  new_df = new_df.drop(columns=['level_0', 'level_1'])
  new_df.columns = ['Sentence']
  new_df['len'] = new_df.Sentence.apply(lambda x: len(x.split()))

  # Обрабатываем текст: переводим слова в нижний регистр, оставляем предложения длиной больше 5 слов, оставляем только альфанумерические символы
  new_df.Sentence = new_df.Sentence.apply(lambda x: x.lower())
  new_df = new_df[new_df.len >=5]
  new_df.Sentence = new_df.Sentence.apply(lambda x: ' '.join(x for x in x.split() if x.isalnum()))

  # Сохраняем обработанный текст отчета
  file_w = file_r.strip('.txt')
  new_df.to_csv(path_or_buf="/content/reports/{}.csv".format(file_w)) 

  return new_df

Загружаем локально сохраненный текстовый файл

In [ ]:
from google.colab import files

uploaded = files.upload() 
filename = [x for x in uploaded.keys()][0]
output_name = filename.replace('.txt','.csv')
print('input: ',filename, 'output: ', output_name)

In [ ]:
df = pd.read_csv(filename, sep='\n',header=None, names=['Sentence'], quoting=csv.QUOTE_NONE)

Обрабатываем и сохраняем текст

In [ ]:
df = clean_df(file_r=filename, df=df)
df.head(2)

Проверяем наличие файла

In [ ]:
%cd /content/reports
!ls

##### Генерация предсказаний

Токенизируем текст, генерируем признаки, используем обученный на обучающей выборке классификатор для предсказания тональности

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [ ]:
tkz = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
def prepare_df(df):
  df = df.drop_duplicates(subset=['Sentence'])
  df.Sentence = df.Sentence.map(lambda x: str(x).lower()) # переводим символы в нижний регистр
  df['tokenized'] = df.Sentence.apply(lambda x: tkz.encode(tkz.tokenize(x)))
  display(df.head())
  print(df.shape)
  return df

In [ ]:
def get_sentiment(df, max_length=50):
  max_len = 0
  long_ls = []
  for index, i in df.tokenized.iteritems():
    if len(i) > max_len:
        max_len = len(i)
    if len(i) > max_length:
      long_ls.append(index)

  print('Максимальная длина токена:', max_len)
  print('Предложения, превышающие ограничение по длине:', len(long_ls))

  df = df.drop(long_ls)
  print(df.shape)

  padded = np.array([i + [0]*(max_length-len(i)) for i in df.tokenized.values])

  attention_mask = np.where(padded != 0, 1, 0)
  print(attention_mask.shape)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  print('-'*20)
  print('Обучение модели')
  print('-'*20) 
  begin = time.time()

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  
  end = time.time()
  print('Обучение завершено')
  print('Обучение заняло:', "{:.3f}".format(round(end-begin, 3)), 'секунд')
  print('-'*20)

  features = last_hidden_states[0][:,0,:].numpy()

  # predicted = list(rf_clf.predict(features))
  predicted = list(lr_clf.predict(features))
  pred_dict = dict()

  for i in set(predicted):
    if i not in pred_dict.keys():
      pred_dict[i] = predicted.count(i)
  
  df['preds'] = predicted
  preds = pd.DataFrame(data=predicted, columns=['Prediction'])

  assert len(df) == len(preds)

  ton = np.mean(predicted)
  neg = pred_dict.get(-1,0) / len(predicted)
  pos = pred_dict.get(1,0) / len(predicted)
  neut = pred_dict.get(0,0) / len(predicted)
  stats = dict(neutral=neut, positive=pos, negative=neg, tonality=ton)
  df_stats = pd.DataFrame(data=stats, index=[0])

  return df, df_stats

Загружаем нефинансовый отчет

In [ ]:
!ls "/content/gdrive/My Drive/"

In [ ]:
# !cp "/content/gdrive/My Drive/NVTK_17.csv" "NVTK_17.csv"

In [ ]:
report_name = 'NVTK_17.csv' # заменяем название отчета
df = pd.read_csv(report_name, header=0,
                  sep=',')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True) # Удаляем лишний столбец

In [ ]:
df.head(2)

In [ ]:
df = prepare_df(df)

Делим отчет на несколько частей, чтобы уложиться в ограничения по памяти при генерации признаков. Лучше использовать не более 500 предложений за одну итерацию.

In [ ]:
df_1 = df[:500]
df_2 = df[500:1000]
df_3 = df[1000:]

Предсказываем тональность и смотрим на получившиеся результаты

In [ ]:
df_final, df_stats = get_sentiment(df_1, max_length=62)

In [ ]:
df_stats

In [ ]:
stats = dict(neutral=neut/total, positive=pos/total, negative=neg/total, tonality=ton)
df_stats = pd.DataFrame(data=stats, index=[0])
df_stats